Make sure to place your cartodb-query.csv from https://data.phila.gov/visualizations/crime-incidents in the data folder to run the project. You can always rerun everything with updated rows of data and the code will handle it

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from datetime import datetime
import matplotlib.pyplot as plt

In [2]:
crimes = pd.read_csv('data\cartodb-query.csv')
data = crimes[["dispatch_date", "dispatch_time","location_block", "text_general_code", "point_x", "point_y"]]

In [3]:
data.head()

,dispatch_date,dispatch_time,location_block,text_general_code,point_x,point_y
0,2023-11-22,23:16:00,3200 BLOCK TEESDALE ST,Thefts,-75.045847,40.040386
1,2023-11-22,23:16:00,900 BLOCK N 16TH ST,Thefts,-75.163144,39.970767
2,2023-11-22,23:15:00,900 BLOCK E GODFREY AV,Motor Vehicle Theft,NaN,NaN
3,2023-11-22,23:04:00,100 BLOCK S 10TH ST,Thefts,-75.156919,39.949558
4,2023-11-22,22:55:00,6800 BLOCK BUSTLETON AV,Motor Vehicle Theft,-75.065641,40.040823


# Feature Preprocessing

Dissect Date and Time into a set of categorical features

In [4]:
data['dispatch_date'] = pd.to_datetime(data['dispatch_date'])
data['dispatch_time'] = pd.to_datetime(data['dispatch_time'], format='%H:%M:%S').dt.time

# Combine date and time into a single DateTime column
data['DateTime'] = pd.to_datetime(data['dispatch_date'].astype(str) + ' ' + data['dispatch_time'].astype(str))

# Extracting features
data['Year'] = data['DateTime'].dt.year
data['Month'] = data['DateTime'].dt.month
data['Day'] = data['DateTime'].dt.day
data['Hour'] = data['DateTime'].dt.hour
data['Minute'] = data['DateTime'].dt.minute
data['Weekday'] = data['DateTime'].dt.weekday  # Monday=0, Sunday=6
data['IsWeekend'] = data['Weekday'].apply(lambda x: 1 if x >= 5 else 0)

C:\Users\Alex Zavalny\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [5]:
data.head()

,dispatch_date,dispatch_time,location_block,text_general_code,point_x,point_y,DateTime,Year,Month,Day,Hour,Minute,Weekday,IsWeekend
0,2023-11-22,23:16:00,3200 BLOCK TEESDALE ST,Thefts,-75.045847,40.040386,2023-11-22 23:16:00,2023,11,22,23,16,2,0
1,2023-11-22,23:16:00,900 BLOCK N 16TH ST,Thefts,-75.163144,39.970767,2023-11-22 23:16:00,2023,11,22,23,16,2,0
2,2023-11-22,23:15:00,900 BLOCK E GODFREY AV,Motor Vehicle Theft,NaN,NaN,2023-11-22 23:15:00,2023,11,22,23,15,2,0
3,2023-11-22,23:04:00,100 BLOCK S 10TH ST,Thefts,-75.156919,39.949558,2023-11-22 23:04:00,2023,11,22,23,4,2,0
4,2023-11-22,22:55:00,6800 BLOCK BUSTLETON AV,Motor Vehicle Theft,-75.065641,40.040823,2023-11-22 22:55:00,2023,11,22,22,55,2,0


Extracted street names and block numbers

In [6]:
#  Extracting street names and block numbers allows for more granular analysis of crime locations.
# While streets don't have an inherent order, the block numbers can provide some ordinal information
# his information can pave the way for more advanced geospatial analysis or clustering based on proximity
data['street_name'] = data['location_block'].str.extract('(BLOCK [A-Z]+)')
data['block_number'] = data['location_block'].str.extract('(\d+) BLOCK')
data = data.drop("location_block", axis=1)

C:\Users\Alex Zavalny\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\pandas\core\frame.py:3599: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item_frame_value(key, value)


In [7]:
data.head()

,dispatch_date,dispatch_time,text_general_code,point_x,point_y,DateTime,Year,Month,Day,Hour,Minute,Weekday,IsWeekend,street_name,block_number
0,2023-11-22,23:16:00,Thefts,-75.045847,40.040386,2023-11-22 23:16:00,2023,11,22,23,16,2,0,BLOCK TEESDALE,3200
1,2023-11-22,23:16:00,Thefts,-75.163144,39.970767,2023-11-22 23:16:00,2023,11,22,23,16,2,0,BLOCK N,900
2,2023-11-22,23:15:00,Motor Vehicle Theft,NaN,NaN,2023-11-22 23:15:00,2023,11,22,23,15,2,0,BLOCK E,900
3,2023-11-22,23:04:00,Thefts,-75.156919,39.949558,2023-11-22 23:04:00,2023,11,22,23,4,2,0,BLOCK S,100
4,2023-11-22,22:55:00,Motor Vehicle Theft,-75.065641,40.040823,2023-11-22 22:55:00,2023,11,22,22,55,2,0,BLOCK BUSTLETON,6800


One Hot Encoding Crime Type, since there is no inherent order in the crime types

In [8]:
data.rename(columns={'text_general_code':'crime_type'}, inplace=True)
data["crime_type"].value_counts()

All Other Offenses                         569310
Other Assaults                             439017
Thefts                                     436749
Vandalism/Criminal Mischief                300212
Theft from Vehicle                         261431
Narcotic / Drug Law Violations             180691
Fraud                                      177343
Burglary Residential                       122780
Aggravated Assault No Firearm              104540
Motor Vehicle Theft                         93932
Robbery No Firearm                          72043
DRIVING UNDER THE INFLUENCE                 66712
Robbery Firearm                             56398
Aggravated Assault Firearm                  48689
Disorderly Conduct                          44178
Weapon Violations                           38732
Burglary Non-Residential                    35931
Other Sex Offenses (Not Commercialized)     21686
Rape                                        17689
Prostitution and Commercialized Vice        15948


In [9]:
# combining Homicide categories together
data["crime_type"] = data["crime_type"].apply(lambda x: "Homicides" if isinstance(x, str) and "Homicide" in x else x)
data["crime_type"].value_counts()

All Other Offenses                         569310
Other Assaults                             439017
Thefts                                     436749
Vandalism/Criminal Mischief                300212
Theft from Vehicle                         261431
Narcotic / Drug Law Violations             180691
Fraud                                      177343
Burglary Residential                       122780
Aggravated Assault No Firearm              104540
Motor Vehicle Theft                         93932
Robbery No Firearm                          72043
DRIVING UNDER THE INFLUENCE                 66712
Robbery Firearm                             56398
Aggravated Assault Firearm                  48689
Disorderly Conduct                          44178
Weapon Violations                           38732
Burglary Non-Residential                    35931
Other Sex Offenses (Not Commercialized)     21686
Rape                                        17689
Prostitution and Commercialized Vice        15948


In [10]:
data.head()

,dispatch_date,dispatch_time,crime_type,point_x,point_y,DateTime,Year,Month,Day,Hour,Minute,Weekday,IsWeekend,street_name,block_number
0,2023-11-22,23:16:00,Thefts,-75.045847,40.040386,2023-11-22 23:16:00,2023,11,22,23,16,2,0,BLOCK TEESDALE,3200
1,2023-11-22,23:16:00,Thefts,-75.163144,39.970767,2023-11-22 23:16:00,2023,11,22,23,16,2,0,BLOCK N,900
2,2023-11-22,23:15:00,Motor Vehicle Theft,NaN,NaN,2023-11-22 23:15:00,2023,11,22,23,15,2,0,BLOCK E,900
3,2023-11-22,23:04:00,Thefts,-75.156919,39.949558,2023-11-22 23:04:00,2023,11,22,23,4,2,0,BLOCK S,100
4,2023-11-22,22:55:00,Motor Vehicle Theft,-75.065641,40.040823,2023-11-22 22:55:00,2023,11,22,22,55,2,0,BLOCK BUSTLETON,6800


# Feature Engineering

Distance of crime to nearest police station

If you get a shape mismatch error, be sure to rerun getPoliceStationDistances.py

In [11]:
import json

min_distances = json.load(open("data\min_distances.json"))["min_distances"]
data["distance_to_nearest_police_station"] = min_distances
data.head()

,dispatch_date,dispatch_time,crime_type,point_x,point_y,DateTime,Year,Month,Day,Hour,Minute,Weekday,IsWeekend,street_name,block_number,distance_to_nearest_police_station
0,2023-11-22,23:16:00,Thefts,-75.045847,40.040386,2023-11-22 23:16:00,2023,11,22,23,16,2,0,BLOCK TEESDALE,3200,0.026524
1,2023-11-22,23:16:00,Thefts,-75.163144,39.970767,2023-11-22 23:16:00,2023,11,22,23,16,2,0,BLOCK N,900,0.010736
2,2023-11-22,23:15:00,Motor Vehicle Theft,NaN,NaN,2023-11-22 23:15:00,2023,11,22,23,15,2,0,BLOCK E,900,inf
3,2023-11-22,23:04:00,Thefts,-75.156919,39.949558,2023-11-22 23:04:00,2023,11,22,23,4,2,0,BLOCK S,100,0.007308
4,2023-11-22,22:55:00,Motor Vehicle Theft,-75.065641,40.040823,2023-11-22 22:55:00,2023,11,22,22,55,2,0,BLOCK BUSTLETON,6800,0.009979


In [12]:
weather = pd.read_csv('data/weather.csv')
# Droping the first two rows
weather_cleaned = weather.drop([0, 1]).reset_index(drop=True)

# Setting the third row as the header
weather_cleaned.columns = weather.iloc[1]

# Dropping the now redundant third row
weather_cleaned = weather_cleaned.drop(2).reset_index(drop=True)

# Displaying the first few rows of the cleaned weather dataframe
weather_cleaned.head()

1,time,temperature_2m_max (°F),temperature_2m_min (°F),temperature_2m_mean (°F),precipitation_sum (mm),precipitation_hours (h)
0,2006-01-01,45.5,30.5,36.5,0.00,0.0
1,2006-01-02,46.9,31.1,38.0,12.60,9.0
2,2006-01-04,42.7,31.5,36.3,0.30,1.0
3,2006-01-05,50.5,37.5,42.5,0.00,0.0
4,2006-01-06,41.3,32.6,37.0,0.00,0.0


In [13]:
# Converting the 'time' column in weather_cleaned to datetime format
weather_cleaned['time'] = pd.to_datetime(weather_cleaned['time'])

# Converting the 'dispatch_date' column in data to datetime format
data['dispatch_date'] = pd.to_datetime(data['dispatch_date'])

# Merging the two dataframes based on the date
merged_data = pd.merge(data, weather_cleaned, left_on='dispatch_date', right_on='time', how='left')
data = merged_data

# Displaying the first few rows of the merged dataframe
data.head()

,dispatch_date,dispatch_time,crime_type,point_x,point_y,DateTime,Year,Month,Day,Hour,...,IsWeekend,street_name,block_number,distance_to_nearest_police_station,time,temperature_2m_max (°F),temperature_2m_min (°F),temperature_2m_mean (°F),precipitation_sum (mm),precipitation_hours (h)
0,2023-11-22,23:16:00,Thefts,-75.045847,40.040386,2023-11-22 23:16:00,2023,11,22,23,...,0,BLOCK TEESDALE,3200,0.026524,NaT,NaN,NaN,NaN,NaN,NaN
1,2023-11-22,23:16:00,Thefts,-75.163144,39.970767,2023-11-22 23:16:00,2023,11,22,23,...,0,BLOCK N,900,0.010736,NaT,NaN,NaN,NaN,NaN,NaN
2,2023-11-22,23:15:00,Motor Vehicle Theft,NaN,NaN,2023-11-22 23:15:00,2023,11,22,23,...,0,BLOCK E,900,inf,NaT,NaN,NaN,NaN,NaN,NaN
3,2023-11-22,23:04:00,Thefts,-75.156919,39.949558,2023-11-22 23:04:00,2023,11,22,23,...,0,BLOCK S,100,0.007308,NaT,NaN,NaN,NaN,NaN,NaN
4,2023-11-22,22:55:00,Motor Vehicle Theft,-75.065641,40.040823,2023-11-22 22:55:00,2023,11,22,22,...,0,BLOCK BUSTLETON,6800,0.009979,NaT,NaN,NaN,NaN,NaN,NaN


In [14]:
unemp = pd.read_csv('data/unemployment rate.csv')
# Converting 'label' column in unemp dataframe to string type
unemp['label'] = unemp['label'].astype(str)

# Extracting month and year from dispatch_date
merged_data['Year-Month'] = merged_data['dispatch_date'].dt.strftime('%Y-%m')

# Merging with unemployment data
merged_data = pd.merge(merged_data, unemp, left_on='Year-Month', right_on='label', how='left')

# If there are any missing values, filling them with the most recent unemployment rate
most_recent_rate = unemp['Unemployment Rate of a Population'].iloc[0]
merged_data['Unemployment Rate of a Population'].fillna(most_recent_rate, inplace=True)

# Dropping unnecessary columns
merged_data.drop(columns=["label", "dispatch_date", "label", "Year-Month"], inplace=True)
merged_data.rename({"Unemployment Rate of a Population": "Unemployment Rate"})
data = merged_data
# Displaying the cleaned dataframe
data.head()

,dispatch_time,crime_type,point_x,point_y,DateTime,Year,Month,Day,Hour,Minute,...,street_name,block_number,distance_to_nearest_police_station,time,temperature_2m_max (°F),temperature_2m_min (°F),temperature_2m_mean (°F),precipitation_sum (mm),precipitation_hours (h),Unemployment Rate of a Population
0,23:16:00,Thefts,-75.045847,40.040386,2023-11-22 23:16:00,2023,11,22,23,16,...,BLOCK TEESDALE,3200,0.026524,NaT,NaN,NaN,NaN,NaN,NaN,5.4
1,23:16:00,Thefts,-75.163144,39.970767,2023-11-22 23:16:00,2023,11,22,23,16,...,BLOCK N,900,0.010736,NaT,NaN,NaN,NaN,NaN,NaN,5.4
2,23:15:00,Motor Vehicle Theft,NaN,NaN,2023-11-22 23:15:00,2023,11,22,23,15,...,BLOCK E,900,inf,NaT,NaN,NaN,NaN,NaN,NaN,5.4
3,23:04:00,Thefts,-75.156919,39.949558,2023-11-22 23:04:00,2023,11,22,23,4,...,BLOCK S,100,0.007308,NaT,NaN,NaN,NaN,NaN,NaN,5.4
4,22:55:00,Motor Vehicle Theft,-75.065641,40.040823,2023-11-22 22:55:00,2023,11,22,22,55,...,BLOCK BUSTLETON,6800,0.009979,NaT,NaN,NaN,NaN,NaN,NaN,5.4


In [15]:
data.drop(columns=["time"], inplace=True)

Taking care of missing values

In [16]:
data.isna().sum()

dispatch_time                              0
crime_type                                 0
point_x                                44933
point_y                                44933
DateTime                                   0
Year                                       0
Month                                      0
Day                                        0
Hour                                       0
Minute                                     0
Weekday                                    0
IsWeekend                                  0
street_name                           271296
block_number                          260650
distance_to_nearest_police_station         0
temperature_2m_max (°F)                17760
temperature_2m_min (°F)                17760
temperature_2m_mean (°F)               18135
precipitation_sum (mm)                 18135
precipitation_hours (h)                15090
Unemployment Rate of a Population          0
dtype: int64

In [17]:
data.dropna(inplace=True)

In [18]:
data.columns

Index(['dispatch_time', 'crime_type', 'point_x', 'point_y', 'DateTime', 'Year',
       'Month', 'Day', 'Hour', 'Minute', 'Weekday', 'IsWeekend', 'street_name',
       'block_number', 'distance_to_nearest_police_station',
       'temperature_2m_max (°F)', 'temperature_2m_min (°F)',
       'temperature_2m_mean (°F)', 'precipitation_sum (mm)',
       'precipitation_hours (h)', 'Unemployment Rate of a Population'],
      dtype='object')

In [19]:
data.drop(["dispatch_time", "DateTime"], axis=1, inplace=True)

In [20]:
data.head()

,crime_type,point_x,point_y,Year,Month,Day,Hour,Minute,Weekday,IsWeekend,street_name,block_number,distance_to_nearest_police_station,temperature_2m_max (°F),temperature_2m_min (°F),temperature_2m_mean (°F),precipitation_sum (mm),precipitation_hours (h),Unemployment Rate of a Population
17539,Robbery Firearm,-75.072610,40.041574,2023,10,13,23,54,4,0,BLOCK LARGE,6600,0.017699,65.5,51.5,57.2,0.00,0.0,5.4
17540,Other Assaults,-75.172952,39.999995,2023,10,13,23,27,4,0,BLOCK N,2900,0.019584,65.5,51.5,57.2,0.00,0.0,5.4
17541,Thefts,-75.108462,39.994303,2023,10,13,23,27,4,0,BLOCK JOYCE,3200,0.028199,65.5,51.5,57.2,0.00,0.0,5.4
17542,Thefts,-75.136074,39.991890,2023,10,13,23,24,4,0,BLOCK N,2700,0.029554,65.5,51.5,57.2,0.00,0.0,5.4
17543,Robbery Firearm,-75.107092,40.032184,2023,10,13,23,19,4,0,BLOCK ADAMS,700,0.038528,65.5,51.5,57.2,0.00,0.0,5.4


In [21]:
data.to_csv("data\data_full_features.csv")